# RAG with Llama 2 and LangChain
Retrieval-Augmented Generation (RAG) is a technique that combines a retriever and a generative language model to deliver accurate response. It involves retrieving relevant information from a large corpus and then generating contextually appropriate responses to queries. Here we use the quantized version of the Llama 2 13B LLM with LangChain to perform generative QA with RAG. The notebook file has been tested in Google Colab with T4 GPU. Please change the runtime type to T4 GPU before running the notebook.

GPT 4.0 Model: openai  Company 2) Llama2 Model :Facebook 3) Gemini : Google

## Install Packages

In [ ]:
!pip install transformers>=4.32.0 optimum>=1.12.0
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
!pip install langchain
!pip install chromadb
!pip install sentence_transformers # ==2.2.2
!pip install unstructured
!pip install pdf2image
!pip install pdfminer.six
!pip install unstructured-pytesseract
!pip install unstructured-inference
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/


## Restart Runtime

In [ ]:
import os
os.kill(os.getpid(), 9)

## Load Llama 2
We will use the quantized version of the LLAMA 2 13B model from HuggingFace for our RAG task.

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

model_name = "TheBloke/Llama-2-13b-Chat-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

gen_cfg = GenerationConfig.from_pretrained(model_name)
gen_cfg.max_new_tokens=512
gen_cfg.temperature=0.0000001 # 0.0
gen_cfg.return_full_text=True
gen_cfg.do_sample=True
gen_cfg.repetition_penalty=1.11

pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=gen_cfg
)

llm = HuggingFacePipeline(pipeline=pipe)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
The cos_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class
The sin_cached attribute will be removed in 4.39. Bear in mind that its contents changed in v4.38. Use the forward method of RoPE from now on instead. It is not used in the `LlamaAttention` class


generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

#### Test LLM with Llama 2 prompt structure and LangChain PromptTemplate

In [ ]:
from textwrap import fill
from langchain.prompts import PromptTemplate

template = """
<s>[INST] <<SYS>>
You are an AI assistant. You are truthful, unbiased and honest in your response.

If you are unsure about an answer, truthfully say "I don't know"
<</SYS>>

{text} [/INST]
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

text = "Explain artificial intelligence in a few lines"
result = llm(prompt.format(text=text))
print(fill(result.strip(), width=100))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<s>[INST] <<SYS>> You are an AI assistant. You are truthful, unbiased and honest in your response.
If you are unsure about an answer, truthfully say "I don't know" <</SYS>>  Explain artificial
intelligence in a few lines [/INST]   Sure! Here is my explanation of artificial intelligence:
Artificial intelligence (AI) refers to the development of computer systems that can perform tasks
that typically require human intelligence, such as learning, problem-solving, and decision-making.
These systems use algorithms and machine learning techniques to analyze data and make predictions or
take actions based on that data. Some examples of AI include natural language processing, image
recognition, and autonomous vehicles. Overall, the goal of AI is to create machines that can think
and act like humans, but with greater speed, accuracy, and consistency.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## RAG from web pages
### A. Create a vectore store for the context/external data
Here, we'll create embedding vectores of the unstructured data loaded from the the source and store them in a vectore store.  

####Load the document

Depending on the type of the source data, we can use the appropriate data loader from LangChain to load the data.



In [ ]:
!pip install pillow_heif

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores.utils import filter_complex_metadata # 'filter_complex_metadata' removes complex metadata that are not in str, int, float or bool format

web_loader = UnstructuredURLLoader(
    urls=["https://en.wikipedia.org/wiki/Solar_System"], mode="elements", strategy="fast",
    )
web_doc = web_loader.load()
updated_web_doc = filter_complex_metadata(web_doc)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


####Split the documents into chunks

Due to the limited size of the context window of an LLM, the data need to be divided into smaller chunks with a text splitter like ``CharacterTextSplitter`` or ``RecursiveCharacterTextSplitter``. In this way, the smaller chunks can be fed into the LLM.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_web_doc = text_splitter.split_documents(updated_web_doc)
len(chunked_web_doc)

905

#### Create a vector database of the chunked documents with HuggingFace embeddings

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

We can either use Chroma or FAISS to create the [Vector Store](https://python.langchain.com/docs/modules/data_connection/vectorstores.html).

In [ ]:
%%time

# Create the vectorized db with FAISS
from langchain.vectorstores import FAISS
db_web = FAISS.from_documents(chunked_web_doc, embeddings)

# Create the vectorized db with Chroma
# from langchain.vectorstores import Chroma
# db_web = Chroma.from_documents(chunked_web_doc, embeddings)

CPU times: user 3.67 s, sys: 25.1 ms, total: 3.69 s
Wall time: 4 s


### B. Use RetrievalQA chain
We instantiate a RetrievalQA chain from LangChain which takes in a retriever, LLM and a chain_type as the input arguments. When the QA chain receives a query, the retriever retrieves information relevent to the query from the vectore store.   The ``chain type = "stuff"`` method stuffs all the retrieved information into context and makes a call to the language model. The LLM then generates the text/response from the retrieved documents. [See information on Langchain Retriver](https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa).

**LLM prompt structure**

We can also pass in the recommended prompt structue for Llama 2 for the QA. In this way, we'd be able to advise our LLM to only use the available context to answer our question. If it cannot find information relevant to our query in the context, it'll **NOT** make up an answer, rather, it would advise that it's unable to find relevant information in the context.

In [ ]:
%%time

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# use the recommended propt style for the LLAMA 2 LLM
prompt_template = """
<s>[INST] <<SYS>>
Use the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.

<</SYS>>

{context}

Question: {question} [/INST]
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
Chain_web = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    # retriever=db.as_retriever(search_type="similarity_score_threshold", search_kwargs={'k': 5, 'score_threshold': 0.8})
    # Similarity Search is the default way to retrieve documents relevant to a query, but we can use MMR by setting search_type = "mmr"
    # k defines how many documents are returned; defaults to 4.
    # score_threshold allows to set a minimum relevance for documents returned by the retriever, if we are using the "similarity_score_threshold" search type.
    # return_source_documents=True, # Optional parameter, returns the source documents used to answer the question
    retriever=db_web.as_retriever(), # (search_kwargs={'k': 5, 'score_threshold': 0.8}),
    chain_type_kwargs={"prompt": prompt},
)
query = "When was the solar system formed?"
result = Chain_web(query)
result

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


CPU times: user 1.96 s, sys: 151 ms, total: 2.12 s
Wall time: 2.15 s


{'query': 'When was the solar system formed?',
 'result': "\n<s>[INST] <<SYS>>\nUse the following context to Answer the question at the end. Do not use any other information. If you can't find the relevant information in the context, just say you don't have enough information to answer the question. Don't try to make up an answer.\n\n<</SYS>>\n\nFormation and evolution of the Solar System\n\nThe Solar System formed 4.568\xa0billion years ago from the gravitational collapse of a region within a large molecular cloud.[f] This initial cloud was likely several light-years across and probably birthed several stars.[8] As is typical of molecular clouds, this one consisted mostly of hydrogen, with some helium, and small amounts of heavier elements fused by previous generations of stars.[9]\n\nSolar System\n\nSolar System\n\nQuestion: When was the solar system formed? [/INST]\n\nBased on the information provided, the Solar System was formed approximately 4.568 billion years ago."}

In [ ]:
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  Formation and
evolution of the Solar System  The Solar System formed 4.568 billion years ago from the
gravitational collapse of a region within a large molecular cloud.[f] This initial cloud was likely
several light-years across and probably birthed several stars.[8] As is typical of molecular clouds,
this one consisted mostly of hydrogen, with some helium, and small amounts of heavier elements fused
by previous generations of stars.[9]  Solar System  Solar System  Question: When was the solar
system formed? [/INST]  Based on the information provided, the Solar System was formed approximately
4.568 billion years ago.


In [ ]:
%%time

query = "How was the solar system formed?"
result = Chain_web(query)
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  Formation and
evolution of the Solar System  The Solar System formed 4.568 billion years ago from the
gravitational collapse of a region within a large molecular cloud.[f] This initial cloud was likely
several light-years across and probably birthed several stars.[8] As is typical of molecular clouds,
this one consisted mostly of hydrogen, with some helium, and small amounts of heavier elements fused
by previous generations of stars.[9]  ^ Woolfson, M. (2000). "The origin and evolution of the solar
system". Astronomy & Geophysics. 41 (1): 12. Bibcode:2000A&G....41a..12W.
doi:10.1046/j.1468-4004.2000.00012.x.  ^ Woolfson, M. (2000). "The origin and evolution of the solar
system". Astronomy & Geophysics. 41 (1)

In [ ]:
%%time

query = "Why do the planets orbit the Sun in the same direction that the Sun is rotating?"
result = Chain_web(query)
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  As a result of
the formation of the Solar System, planets and most other objects orbit the Sun in the same
direction that the Sun is rotating. That is, counter-clockwise, as viewed from above Earth's north
pole.[168] There are exceptions, such as Halley's Comet.[169] Most of the larger moons orbit their
planets in prograde direction, matching the planetary rotation; Neptune's moon Triton is the largest
to orbit in the opposite, retrograde manner.[170] Most larger objects rotate around their own axes
in the prograde direction relative to their orbit, though the rotation of Venus is retrograde.[171]
The planets and other large objects in orbit around the Sun lie near the plane of Earth's orbit,
known as the ecl

In [ ]:
%%time

query = "What are planets of the solar system composed of? Give a detailed response."
result = Chain_web(query)
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  The four
terrestrial or inner planets have dense, rocky compositions, few or no moons, and no ring systems.
They are composed largely of refractory minerals such as silicates—which form their crusts and
mantles—and metals such as iron and nickel which form their cores. Three of the four inner planets
(Venus, Earth and Mars) have atmospheres substantial enough to generate weather; all have impact
craters and tectonic surface features, such as rift valleys and volcanoes. The term inner planet
should not be confused with inferior planet, which designates those planets that are closer to the
Sun than Earth (i.e. Mercury and Venus).[40]  ^ a b c According to IAU definitions, objects orbiting
the Sun are classified

Let's test our LLM with a query that is not relevant to the context. The model should respond that it does not have enough information to respond to this query.

In [ ]:
%%time

query = "How does the tranformers architecture work?"
result = Chain_web(query)
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  fused into
Composition  Portals:  orbiter.  Question: How does the tranformers architecture work? [/INST]
Based on the context provided, I cannot answer the question about how the transformers architecture
works. The context only mentions "tranformers" in the phrase "fused into," but it does not provide
any information about what transformers are or how they work. Without more information, I cannot
provide a meaningful answer to this question.
CPU times: user 4.95 s, sys: 103 ms, total: 5.05 s
Wall time: 5.11 s


**The model responded as expected. It does not have the information to answer the question!**

## RAG from PDF Files

Download pdf files

In [ ]:
!gdown "https://github.com/muntasirhsn/datasets/raw/main/Solar-System-Wikipedia.pdf" # this is just a pdf print of the Solar System page on Wikipedia!

Downloading...
From: https://github.com/muntasirhsn/datasets/raw/main/Solar-System-Wikipedia.pdf
To: /content/Solar-System-Wikipedia.pdf
100% 4.49M/4.49M [00:00<00:00, 18.8MB/s]


Load PDF Files

In [ ]:
pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader("/content/Solar-System-Wikipedia.pdf")
pdf_doc = pdf_loader.load()
updated_pdf_doc = filter_complex_metadata(pdf_doc)

Spit the document into chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunked_pdf_doc = text_splitter.split_documents(updated_pdf_doc)
len(chunked_pdf_doc)

226

Create the vector store

In [ ]:
%%time
db_pdf = FAISS.from_documents(chunked_pdf_doc, embeddings)

CPU times: user 4.87 s, sys: 8.14 ms, total: 4.88 s
Wall time: 4.86 s


RAG with RetrievalQA

In [ ]:
%%time

Chain_pdf = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db_pdf.as_retriever(),
    chain_type_kwargs={"prompt": prompt},
)
query = "Why do the planets orbit the Sun in the same direction that the Sun is rotating?"
result = Chain_pdf(query)
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  synchronous
rotat ion, with one face permanently turne d toward their parent. The four giant planets have
planetary rings, thin bands of tiny particles that orbit them in unison.[29] As a resu lt of the
formation of the Solar System , plan ets and most other objects orbit the Sun in the same direction
that the Sun is rotating. That is, counter-clockwise, as viewed from above Earth's north pole.[30]
There are exceptions, such as Halley's Comet .[31] Most of the large r moons orbit their planets in
prograde  direction, matching the planetary rotation; Neptune's moon Triton  is the largest to orbit
in the opposite, retrograde  manner.[32] Most larger objects rotate around their own axes in the
prograde direction

In [ ]:
%%time

query = "Can you explain what swift code is?"
result = Chain_pdf(query)
print(fill(result['result'].strip(), width=100))

<s>[INST] <<SYS>> Use the following context to Answer the question at the end. Do not use any other
information. If you can't find the relevant information in the context, just say you don't have
enough information to answer the question. Don't try to make up an answer.  <</SYS>>  4956
(https://www .worldcat.org/issn/0935-4956) . S2CID  7683815  (https://api.semanticscholar .o
rg/CorpusID:7683815) . Archived  (https://web.archive.org/web/20220420161227/https://idp.spri
nger.com/favicon.ico)  from the original on 20 April 2022 . Retrieved 9 March  2022 .
orldcat.org/oclc/961476196) . Houston, Texas: OpenStax . ISBN  978-1-947-17224-1 . OCLC  961476196
(https://www .worldcat.org/oclc/961476196) . Archived  (https://web.archive.or
g/web/20200719090803/http://worldcat.org/oclc/961476196)  from the original on 19 July 2020 .
Retrieved 9 March  2022 .  Bibcode :2022ApJ...937L..32S
(https://ui.adsabs.harvard.edu/abs/2022ApJ...937L..32S) . doi:10.3847/2041-8213/ac9120
(https://doi.org/10.3847%